# Experiments Notebook

In [ ]:
%run problem_creator.ipynb
import math, heapq, time, pandas as pd, matplotlib.pyplot as plt


## Algorithms

In [ ]:
def dijkstra(graph, start=0):
    dist={v:math.inf for v in graph}; parent={v:None for v in graph}
    dist[start]=0; pq=[(0,start)]
    while pq:
        d,u=heapq.heappop(pq)
        if d>dist[u]: continue
        for v,w in graph[u]:
            if d+w<dist[v]:
                dist[v]=d+w; parent[v]=u; heapq.heappush(pq,(dist[v],v))
    return dist,parent

def bellman_ford(graph, start=0):
    dist={v:math.inf for v in graph}; parent={v:None for v in graph}
    dist[start]=0
    for _ in range(len(graph)-1):
        upd=False
        for u in graph:
            for v,w in graph[u]:
                if dist[u]+w<dist[v]:
                    dist[v]=dist[u]+w; parent[v]=u; upd=True
        if not upd: break
    for u in graph:
        for v,w in graph[u]:
            if dist[u]+w<dist[v]: raise ValueError("Negative cycle")
    return dist,parent

def reconstruct_path(parent, target):
    path=[]
    while target is not None:
        path.append(target); target=parent[target]
    return path[::-1]


## Negative Edge Check

In [ ]:
def has_negative_edge(graph):
    return any(w<0 for edges in graph.values() for _,w in edges)


## Experiment Runner

In [ ]:
def run_single_experiment(n,density,noise,neg_prob,seed):
    g=generate_random_graph(n,density,noise,neg_prob,seed)
    start=0; t0=time.time()
    if has_negative_edge(g):
        algo="Bellman-Ford"; dist,parent=bellman_ford(g,start)
    else:
        algo="Dijkstra"; dist,parent=dijkstra(g,start)
    return {"n":n,"density":density,"noise":noise,"neg_prob":neg_prob,"seed":seed,"algorithm":algo,"runtime":time.time()-t0}, g, dist, parent


## Experiment Grid

In [ ]:
sizes=[20,50,100]; densities=[0.2,0.5,0.8]; noises=[0.0,0.2]; neg_probs=[0.0,0.1]; seeds=[0,1,2]
rows=[]
for n in sizes:
    for d in densities:
        for nz in noises:
            for ng in neg_probs:
                for s in seeds:
                    result,_,_,_=run_single_experiment(n,d,nz,ng,s)
                    rows.append(result)
df=pd.DataFrame(rows)
df.head()


## Summary

In [ ]:
summary=df.groupby(["n","density","noise","neg_prob","algorithm"])["runtime"].mean().reset_index()
summary


## Plot Runtime vs Size

In [ ]:
plt.figure(figsize=(8,5))
for algo in summary["algorithm"].unique():
    sub=summary[(summary["algorithm"]==algo)&(summary["density"]==0.5)&(summary["noise"]==0.0)&(summary["neg_prob"]==0.0)]
    if not sub.empty:
        plt.plot(sub["n"], sub["runtime"], marker="o", label=algo)
plt.xlabel("Size"); plt.ylabel("Runtime"); plt.title("Runtime vs Size"); plt.legend(); plt.grid(True); plt.show()


## Example Path Reconstruction

In [ ]:
demo_graph=generate_random_graph(8, density=0.4, noise=0.1, negative_prob=0.0, seed=123)
dist,parent=dijkstra(demo_graph,0)
target=5
path=reconstruct_path(parent,target)
print("Demo Graph:"); pretty_print_graph(demo_graph)
print("\nDistances:", dist)
print(f"\nPath 0 -> {target}:", path)


## Results Summary
This section shows example experiment results.

In [ ]:
import pandas as pd
# Example subset results
df_results = pd.DataFrame({
    'n':[20,20,50],
    'density':[0.5,0.5,0.5],
    'noise':[0.0,0.0,0.0],
    'neg_prob':[0.0,0.1,0.0],
    'algo':['Dijkstra','Bellman-Ford','Dijkstra'],
    'runtime':[0.00002,0.00004,0.00008]
})
df_results